In [ ]:
import os
import glob
import pandas as pd
from codebase.running_models import run_datasets, split_into_bins, distance_metrics, model_data,  get_name
from codebase.generic_processing import dataset_name_generator

dataset_name = dataset_name_generator(noise=0.1)
# run_dataset = 'friedman_10_noise_' + str(.1) + '_window_size_2'
directory = 'toy_datasets/' + dataset_name
result = glob.glob(directory + '/*.csv')
result.sort()
dataset_names = result
features = ['0', '1', '2', '3']
for train_dataset_name in dataset_names:
    for target_dataset_name in dataset_names:
        if train_dataset_name != target_dataset_name:
            df_train = pd.read_csv(train_dataset_name)
            df_target = pd.read_csv(target_dataset_name)

            reporter_objects = []
            reporter_object = model_data(train_dataset_name, target_dataset_name)
            for _ in range(1000):
                reporter_objects.append(model_data(get_name(train_dataset_name), get_name(target_dataset_name)))

            i = 0
            apply_processing_posibilities = [False]

            for apply_preprocessing in apply_processing_posibilities:
                for feature in features:
                    count, mean, std, bins = split_into_bins(df_train, column=feature)
                    reporter_objects[i].std_train.append(std)
                    reporter_objects[i].mean_train.append(mean)
                    reporter_objects[i].count_train.append(count)
                    reporter_objects[i].bins_train.append(bins)

                    count, mean, std, bins = split_into_bins(df_target, column=feature)
                    reporter_objects[i].std_test.append(std)
                    reporter_objects[i].mean_test.append(mean)
                    reporter_objects[i].count_test.append(count)
                    reporter_objects[i].bins_test.append(bins)

                    kl_divergence, wasserstein_distance_result, hellinger_distance, ks_test_result = distance_metrics(
                        df_train, df_target, column=feature)
                    reporter_objects[i].kl_divergence.append(kl_divergence)
                    reporter_objects[i].wasserstein_distance.append(wasserstein_distance_result)
                    reporter_objects[i].hellinger_distance.append(hellinger_distance)
                    reporter_objects[i].ks_test.append(ks_test_result)

                reporter_objects[i] = run_datasets(df_train, df_target, target='target',
                                                   apply_preprocessing=apply_preprocessing,
                                                   reporter_object=reporter_objects[i])
                # reporter_objects[i].print_model_details()
                i += 1
            for j in range(i):
                if not os.path.exists('toy_datasets/ran_datasets/'):
                    os.makedirs('toy_datasets/ran_datasets/')
                # reporter_objects[j].print_model_details()
                reporter_objects[j].output_to_csv('toy_datasets/ran_datasets/' + dataset_name + '.csv', features)
print('Models ran')
